In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
hs2_codes =  pd.read_csv(r"C:\Users\Admin\Downloads\hs2_codes_corrected.csv")

In [3]:
hs2_codes

,HS2,Description
0,1,Live animals
1,2,Meat and edible meat offal
2,3,Fish & crustaceans
3,4,Dairy produce
4,5,Products of animal origin
...,...,...
93,95,Miscellaneous manufactured articles
94,96,"Works of art, collectors' pieces"
95,97,Special transactions
96,98,Special classification provisions


In [4]:
hs6_codes = pd.read_csv(r"C:\Users\Admin\Desktop\Exports_project\product_codes_HS17_V202501.csv")

In [5]:
hs6_codes = hs6_codes.rename(columns = {'code' : 'HS6',
                                       'description' : 'Description'})

In [6]:
hs6_codes

,HS6,Description
0,10121,"Horses: live, pure-bred breeding animals"
1,10129,"Horses: live, other than pure-bred breeding an..."
2,10130,Asses: live
3,10190,Mules and hinnies: live
4,10221,"Cattle: live, pure-bred breeding animals"
...,...,...
5379,970300,"Sculptures and statuary: original, in any mate..."
5380,970400,"Stamps, postage or revenue: stamp-postmarks, f..."
5381,970500,Collections and collectors' pieces: of zoologi...
5382,970600,Antiques: of an age exceeding one hundred years


In [7]:
hs2_codes['HS2'] = hs2_codes['HS2'].astype(str)  # Convert to string if not already

# Add '0' as prefix for 5-digit values
hs2_codes['HS2'] = hs2_codes['HS2'].apply(lambda x: '0' + x if len(x) == 1 else x)

# Assuming HS6_desc is your DataFrame and 'code' is the column with HS6 values
hs6_codes['HS6'] = hs6_codes['HS6'].astype(str)  # Convert to string if not already

# Add '0' as prefix for 5-digit values
hs6_codes['HS6'] = hs6_codes['HS6'].apply(lambda x: '0' + x if len(x) == 5 else x)

In [8]:
hs2_codes

,HS2,Description
0,01,Live animals
1,02,Meat and edible meat offal
2,03,Fish & crustaceans
3,04,Dairy produce
4,05,Products of animal origin
...,...,...
93,95,Miscellaneous manufactured articles
94,96,"Works of art, collectors' pieces"
95,97,Special transactions
96,98,Special classification provisions


In [9]:
hs6_codes

,HS6,Description
0,010121,"Horses: live, pure-bred breeding animals"
1,010129,"Horses: live, other than pure-bred breeding an..."
2,010130,Asses: live
3,010190,Mules and hinnies: live
4,010221,"Cattle: live, pure-bred breeding animals"
...,...,...
5379,970300,"Sculptures and statuary: original, in any mate..."
5380,970400,"Stamps, postage or revenue: stamp-postmarks, f..."
5381,970500,Collections and collectors' pieces: of zoologi...
5382,970600,Antiques: of an age exceeding one hundred years


In [10]:
df_dict = {}
for file in os.listdir(os.getcwd()):
    if file.startswith('BACI'):
        filename = file.split('_')
        print(filename[2])
        
        df = pd.read_csv(file, dtype={"k": str})
        df = df.rename(columns={
            't': 'year',
            'i': 'exporter',
            'j': 'importer',
            'k': 'HS6',
            'v': 'value(thousands USD)',
            'q': 'quantity(in metric tons)'
        })
        
        # Ensure 6-digit product codes
        df["HS6"] = df["HS6"].str.zfill(6)

        # Extract HS2 and HS4 codes
        df["HS2"] = df["HS6"].str[:2]  # First 2 digits
        df["HS4"] = df["HS6"].str[:4]

        # Merge with HS2 and HS6 description data
        df = df.merge(hs2_codes, on="HS2", how="left").rename(columns={"Description": "HS2_desc"})
        df = df.merge(hs6_codes, on="HS6", how="left").rename(columns={"Description": "HS6_desc"})
        
        df_dict[filename[2]] = df  # Store in dictionary with year as key

Y2017
Y2018
Y2019
Y2020
Y2021
Y2022
Y2023


In [24]:
df_dict['Y2017'][(df_dict['Y2017']['exporter'] == 156) & (df_dict['Y2017']['importer'] == 842)]

,year,exporter,importer,HS6,value(thousands USD),quantity(in metric tons),HS2,HS4,HS2_desc,HS6_desc
1775471,2017,156,842,010129,910.276,41.240,01,0101,Live animals,"Horses: live, other than pure-bred breeding an..."
1775472,2017,156,842,010611,32834.903,44.283,01,0106,Live animals,"Mammals: live, primates"
1775473,2017,156,842,010612,140.000,1.307,01,0106,Live animals,"Mammals: live, whales, dolphins and porpoises ..."
1775474,2017,156,842,010619,119.262,0.587,01,0106,Live animals,"Mammals: live, other than primates, whales, do..."
1775475,2017,156,842,010620,14.003,0.157,01,0106,Live animals,Reptiles: live (including snakes and turtles)
...,...,...,...,...,...,...,...,...,...,...
1780161,2017,156,842,970200,1051.572,1.145,97,9702,Special transactions,"Engravings, prints and lithographs: original"
1780162,2017,156,842,970300,30367.043,NaN,97,9703,Special transactions,"Sculptures and statuary: original, in any mate..."
1780163,2017,156,842,970400,641.883,4.449,97,9704,Special transactions,"Stamps, postage or revenue: stamp-postmarks, f..."
1780164,2017,156,842,970500,42798.468,835.332,97,9705,Special transactions,Collections and collectors' pieces: of zoologi...


In [22]:
for key in df_dict.keys():
    print(len(df_dict[key][(df_dict[key]['exporter'] == 156) & (df_dict[key]['importer'] == 842)]))

4695
4724
4669
4619
4629
4668
4619


In [25]:
#master df

In [39]:
master_df.to_csv('filtered_data.csv',index=False)

In [38]:
import pandas as pd
import numpy as np
import pymysql

# Step 1: Filter and concatenate all matching DataFrames
master_df = pd.DataFrame()
for key in df_dict.keys():
    filtered_df = df_dict[key][(df_dict[key]['exporter'] == 156) & (df_dict[key]['importer'] == 842)]
    master_df = pd.concat([master_df, filtered_df], ignore_index=True)
    
# Clean value column (just drop decimals)
if 'value(thousands USD)' in master_df.columns:
    master_df['value(thousands USD)'] = master_df['value(thousands USD)'].fillna(0).astype(int)

# Clean quantity column (always round UP if > 0)
if 'quantity(in metric tons)' in master_df.columns:
    master_df['quantity(in metric tons)'] = master_df['quantity(in metric tons)'].fillna(0)
    master_df['quantity(in metric tons)'] = np.ceil(master_df['quantity(in metric tons)']).astype(int)

# Step 2: MySQL helper functions (already defined)
def create_table_from_df(cursor, table_name, df):
    dtype_mapping = {
        'int64': 'BIGINT',
        'float64': 'DOUBLE',
        'object': 'TEXT'
    }

    columns = []
    for col in df.columns:
        sql_type = dtype_mapping.get(str(df[col].dtype), 'TEXT')
        columns.append(f"`{col}` {sql_type}")
    columns_sql = ", ".join(columns)

    cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")
    create_query = f"CREATE TABLE `{table_name}` ({columns_sql})"
    cursor.execute(create_query)

def insert_df_in_chunks(cursor, table_name, df, chunk_size=1000):
    df_cleaned = df.replace([np.nan, np.inf, -np.inf], [None, None, None])
    cols = ", ".join([f"`{col}`" for col in df.columns])
    placeholders = ", ".join(["%s"] * len(df.columns))
    insert_query = f"INSERT INTO `{table_name}` ({cols}) VALUES ({placeholders})"

    for start in range(0, len(df_cleaned), chunk_size):
        end = start + chunk_size
        chunk = df_cleaned.iloc[start:end]
        cursor.executemany(insert_query, chunk.values.tolist())

# Step 3: Connect to MySQL
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='1234',
    database='export_data',
    charset='utf8mb4'
)
cursor = conn.cursor()

# Step 4: Save to MySQL
table_name = 'filtered_export_data'
create_table_from_df(cursor, table_name, master_df)
insert_df_in_chunks(cursor, table_name, master_df, chunk_size=1000)

# Step 5: Commit and close
conn.commit()
cursor.close()
conn.close()

In [ ]:
#all df

In [12]:
import pymysql
import pandas as pd
import numpy as np

# Your df_dict — example structure
# df_dict = {'Y2017': df1, 'Y2018': df2, ...}

# Chunk size for big data insertions
chunk_size = 10000

def create_table_from_df(cursor, table_name, df):
    # Mapping pandas dtype to SQL
    dtype_mapping = {
        'int64': 'BIGINT',
        'float64': 'DOUBLE',
        'object': 'TEXT'
    }

    columns = []
    for col in df.columns:
        sql_type = dtype_mapping.get(str(df[col].dtype), 'TEXT')
        columns.append(f"`{col}` {sql_type}")
    columns_sql = ", ".join(columns)

    cursor.execute(f"DROP TABLE IF EXISTS `{table_name}`")
    create_query = f"CREATE TABLE `{table_name}` ({columns_sql})"
    cursor.execute(create_query)

def insert_df_in_chunks(cursor, table_name, df):
    # Replace NaN, inf, -inf with None for MySQL compatibility
    df_cleaned = df.replace([np.nan, np.inf, -np.inf], [None, None, None])

    cols = ", ".join([f"`{col}`" for col in df.columns])
    placeholders = ", ".join(["%s"] * len(df.columns))
    insert_query = f"INSERT INTO `{table_name}` ({cols}) VALUES ({placeholders})"

    for start in range(0, len(df_cleaned), chunk_size):
        end = start + chunk_size
        chunk = df_cleaned.iloc[start:end]
        cursor.executemany(insert_query, chunk.values.tolist())

# Connect to MySQL
conn = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='1234',
    database='export_data',
    charset='utf8mb4'
)
cursor = conn.cursor()

# Process each DataFrame
for table_name, df in df_dict.items():
    print(f"📦 Processing {table_name}: {df.shape}")
    create_table_from_df(cursor, table_name, df)
    insert_df_in_chunks(cursor, table_name, df)
    conn.commit()
    print(f"✅ Saved {table_name} to MySQL")

conn.close()

📦 Processing Y2017: (9946668, 10)
✅ Saved Y2017 to MySQL
📦 Processing Y2018: (10791592, 10)
✅ Saved Y2018 to MySQL
📦 Processing Y2019: (11194511, 10)
✅ Saved Y2019 to MySQL
📦 Processing Y2020: (10959127, 10)
✅ Saved Y2020 to MySQL
📦 Processing Y2021: (11574294, 10)
✅ Saved Y2021 to MySQL
📦 Processing Y2022: (11612508, 10)
✅ Saved Y2022 to MySQL
📦 Processing Y2023: (11474853, 10)
✅ Saved Y2023 to MySQL
